In [1]:
!pip install transformers datasets accelerate evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [15]:
!pip install bitsandbytes accelerate scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Load the Teacher (Already fine-tuned on SST-2)
teacher_id = "textattack/bert-base-uncased-SST-2"
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_id)
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_id)

# 2. Load the Student (Smaller, generic DistilBERT)
student_id = "distilbert-base-uncased"
student_model = AutoModelForSequenceClassification.from_pretrained(
    student_id,
    num_labels=2,
    id2label=teacher_model.config.id2label,
    label2id=teacher_model.config.label2id
)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
teacher_model.to(device)
student_model.to(device)

print(f"Teacher parameters: {teacher_model.num_parameters():,}")
print(f"Student parameters: {student_model.num_parameters():,}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Teacher parameters: 109,483,778
Student parameters: 66,955,010


In [3]:
from transformers import Trainer, TrainingArguments

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.temperature = temperature
        self.alpha = alpha
        # Freeze teacher weights (we only learn from them, we don't update them)
        self.teacher_model.eval()
        self.teacher_model.requires_grad_(False)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # 1. Forward pass student
        outputs_student = model(**inputs)
        student_logits = outputs_student.logits

        # 2. Forward pass teacher (with no gradient tracking for efficiency)
        with torch.no_grad():
            outputs_teacher = self.teacher_model(**inputs)
            teacher_logits = outputs_teacher.logits

        # 3. Calculate "Dark Knowledge" Loss (KL Divergence)
        # We soften the logits using the Temperature (T)
        loss_distill = F.kl_div(
            F.log_softmax(student_logits / self.temperature, dim=1),
            F.softmax(teacher_logits / self.temperature, dim=1),
            reduction='batchmean',
        ) * (self.temperature ** 2)

        # 4. Calculate Standard Loss (Cross Entropy with Ground Truth)
        # Note: 'labels' are automatically handled by the model's internal loss calculation if present
        loss_ce = outputs_student.loss

        # 5. Combine them
        # alpha controls how much we trust the hard labels vs the teacher
        loss = (self.alpha * loss_ce) + ((1 - self.alpha) * loss_distill)

        return (loss, outputs_student) if return_outputs else loss

In [5]:
from datasets import load_dataset
import evaluate
import numpy as np

# Load Data
dataset = load_dataset("glue", "sst2")

# Tokenize
def tokenize_function(examples):
    return teacher_tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define Metrics
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./distilled-bert-sst2",
    per_device_train_batch_size=32,
    num_train_epochs=3,
    learning_rate=2e-5,
    eval_strategy="epoch",
    logging_steps=50,
)

# Initialize our Custom Trainer
trainer = DistillationTrainer(
    model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=teacher_tokenizer,
    compute_metrics=compute_metrics,
    temperature=4.0,  # Soften the probability distribution
    alpha=0.5         # 50% Hard Labels, 50% Teacher Knowledge
)

# Train!
trainer.train()

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

/tmp/ipython-input-3687500800.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: harsha90145 (harsha90145-university-of-alabama-at-birmingham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.263400,0.346283,0.900229
2,0.166400,0.335335,0.900229
3,0.098100,0.333426,0.903670


TrainOutput(global_step=6315, training_loss=0.24634105479632307, metrics={'train_runtime': 3400.9809, 'train_samples_per_second': 59.408, 'train_steps_per_second': 1.857, 'total_flos': 6691160124062208.0, 'train_loss': 0.24634105479632307, 'epoch': 3.0})

In [12]:
student_model.save_pretrained("./distilled-bert-sst2")

In [13]:
import torch
import torch.nn.functional as F

def predict(text, model, tokenizer, device="cpu"):
    # 1. Prepare the model
    model.to(device)
    model.eval()

    # 2. Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)

    # --- THE FIX IS HERE ---
    # BERT tokenizers create 'token_type_ids', but DistilBERT crashes if it sees them.
    # We simply remove them from the dictionary if they exist.
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    # Move remaining inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 3. Run Inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 4. Convert to Probabilities
    probabilities = F.softmax(logits, dim=1)

    # 5. Format Output
    prediction_id = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0][prediction_id].item()

    label_map = {0: "NEGATIVE", 1: "POSITIVE"}
    label_name = label_map[prediction_id]

    return f"Label: {label_name} | Confidence: {confidence:.2%}"

# --- Re-run Test Cases ---
# Make sure model is on CPU for this quick test
student_model.to("cpu")

sample_1 = "The movie was overly long and the plot was confusing."
sample_2 = "An absolute masterpiece that I would watch again in a heartbeat."
sample_3 = "It was okay, not great but not terrible either."

print(f"Input: '{sample_1}'\n -> {predict(sample_1, student_model, teacher_tokenizer)}")
print(f"\nInput: '{sample_2}'\n -> {predict(sample_2, student_model, teacher_tokenizer)}")
print(f"\nInput: '{sample_3}'\n -> {predict(sample_3, student_model, teacher_tokenizer)}")

Input: 'The movie was overly long and the plot was confusing.'
 -> Label: NEGATIVE | Confidence: 99.89%

Input: 'An absolute masterpiece that I would watch again in a heartbeat.'
 -> Label: POSITIVE | Confidence: 99.97%

Input: 'It was okay, not great but not terrible either.'
 -> Label: POSITIVE | Confidence: 94.06%


In [25]:
# !pip install transformers datasets accelerate evaluate torch bitsandbytes scipy numpy

import torch
import time
import os
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig
)

# ==========================================
# 1. SETUP
# ==========================================
student_path = "./distilled_student_saved"  # Change this if your model is elsewhere
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Main Hardware: {device}")

# Check if model exists, otherwise warn user
if not os.path.exists(student_path):
    print(f"WARNING: Path '{student_path}' not found. Using generic DistilBERT for demo.")
    student_path = "distilbert-base-uncased-finetuned-sst-2-english"

# Load Data
dataset = load_dataset("glue", "sst2")
val_dataset = dataset["validation"]
metric = evaluate.load("accuracy")
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")

# ==========================================
# 2. BENCHMARK FUNCTION (With Bug Fix)
# ==========================================
def run_benchmark(model, name, device_type, dataset):
    print(f"\n--- Benchmarking: {name} ({device_type}) ---")

    # A. Handle Device Placement
    try:
        if device_type == "cuda" and not hasattr(model, "hf_device_map"):
            model.to("cuda")
        elif device_type == "cpu":
            model.to("cpu")
    except:
        pass # 4-bit models manage their own device map

    model.eval()

    # B. Measure Size
    try:
        torch.save(model.state_dict(), "temp.p")
        size_mb = os.path.getsize("temp.p") / (1024 * 1024)
        os.remove("temp.p")
    except:
        size_mb = 0.0 # 4-bit models/quantized models often fail state_dict save

    # C. Inference Loop
    latencies = []
    print(f"Evaluating {len(dataset)} items...", end="")

    for i, example in enumerate(dataset):
        inputs = tokenizer(example["sentence"], return_tensors="pt", truncation=True, max_length=128)

        # --- CRITICAL FIX FOR DISTILBERT ---
        if "token_type_ids" in inputs:
            # Check strictly if model class contains 'DistilBert'
            if "DistilBert" in type(model).__name__ or "DistilBert" in getattr(model.config, "architectures", [""])[0]:
                del inputs["token_type_ids"]

        # Move to device
        if device_type == "cuda":
            inputs = {k: v.cuda() for k, v in inputs.items()}

        # Timing
        start = time.time()
        with torch.no_grad():
            outputs = model(**inputs)
        end = time.time()

        latencies.append((end - start) * 1000)

        # Accuracy
        preds = torch.argmax(outputs.logits, dim=-1)
        metric.add(prediction=preds, reference=example["label"])

        if i % 200 == 0: print(".", end="")

    final_acc = metric.compute()['accuracy'] * 100
    avg_lat = np.mean(latencies)

    print(" Done.")
    return {"Model": name, "Type": device_type, "Acc": final_acc, "Size": size_mb, "Lat": avg_lat}

# ==========================================
# 3. RUN EVALUATIONS
# ==========================================
results_table = []

# A. TEACHER (BERT) - Baseline
print("\nLoading Teacher...")
teacher = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")
results_table.append(run_benchmark(teacher, "Teacher (BERT)", "cuda", val_dataset))
del teacher
torch.cuda.empty_cache()

# B. STUDENT (FP32) - Standard Distillation Result
print("\nLoading Student (FP32)...")
student_fp = AutoModelForSequenceClassification.from_pretrained(student_path)
results_table.append(run_benchmark(student_fp, "Student (FP32)", "cuda", val_dataset))

# C. STUDENT (INT8) - CPU Speed Optimized
print("\nQuantizing Student (INT8)...")
student_cpu = student_fp.to("cpu")
student_int8 = torch.quantization.quantize_dynamic(
    student_cpu, {torch.nn.Linear}, dtype=torch.qint8
)
results_table.append(run_benchmark(student_int8, "Student (INT8)", "cpu", val_dataset))
del student_fp, student_int8
torch.cuda.empty_cache()

# D. STUDENT (4-BIT) - VRAM Optimized (BitsAndBytes)
print("\nLoading Student (4-bit)...")
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    student_4bit = AutoModelForSequenceClassification.from_pretrained(
        student_path, quantization_config=bnb_config, device_map="auto"
    )
    results_table.append(run_benchmark(student_4bit, "Student (4-bit)", "cuda", val_dataset))
except ImportError:
    print("Skipping 4-bit eval: bitsandbytes not installed.")

# E. BASELINE (Raw DistilBERT) - The Control Group
# This model has NOT been fine-tuned on SST-2. Accuracy should be ~50%.
print("\nLoading Raw DistilBERT (Baseline)...")
raw_student = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)
# We run this on CUDA to be fair to the other FP32 models
results_table.append(run_benchmark(raw_student, "DistilBERT (Raw)", "cuda", val_dataset))
del raw_student
torch.cuda.empty_cache()

# ==========================================
# 4. FINAL TABLE
# ==========================================
print("\n" + "="*80)
print(f"{'Model':<20} | {'Type':<6} | {'Acc %':<6} | {'Size MB':<8} | {'Lat ms':<8} | {'Notes'}")
print("-" * 80)

for r in results_table:
    note = ""
    if "4-bit" in r['Model']: note = "Low VRAM"
    elif "INT8" in r['Model']: note = "Fastest CPU"
    elif "Teacher" in r['Model']: note = "Teacher"
    elif "Raw" in r['Model']: note = "Untrained Base"
    else: note = "Distilled Student"

    print(f"{r['Model']:<20} | {r['Type']:<6} | {r['Acc']:<6.2f} | {r['Size']:<8.1f} | {r['Lat']:<8.2f} | {note}")
print("="*80)

Main Hardware: cuda

Loading Teacher...

--- Benchmarking: Teacher (BERT) (cuda) ---
Evaluating 872 items........ Done.

Loading Student (FP32)...

--- Benchmarking: Student (FP32) (cuda) ---
Evaluating 872 items........ Done.

Quantizing Student (INT8)...


/tmp/ipython-input-2015461079.py:115: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  student_int8 = torch.quantization.quantize_dynamic(



--- Benchmarking: Student (INT8) (cpu) ---
Evaluating 872 items........

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Done.

Loading Student (4-bit)...
Skipping 4-bit eval: bitsandbytes not installed.

Loading Raw DistilBERT (Baseline)...

--- Benchmarking: DistilBERT (Raw) (cuda) ---
Evaluating 872 items........ Done.

Model                | Type   | Acc %  | Size MB  | Lat ms   | Notes
--------------------------------------------------------------------------------
Teacher (BERT)       | cuda   | 92.43  | 417.7    | 11.86    | Teacher
Student (FP32)       | cuda   | 90.37  | 255.4    | 4.36     | Distilled Student
Student (INT8)       | cpu    | 90.94  | 132.3    | 22.11    | Fastest CPU
DistilBERT (Raw)     | cuda   | 49.08  | 255.4    | 4.29     | Untrained Base


In [21]:
from huggingface_hub import notebook_login

# This will create a widget where you paste your token
notebook_login()

In [22]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 1. Configuration
# =====================================================
local_model_path = "./distilled_student_saved"  # Where we saved it earlier
repo_name = "distilbert-sst2-student"         # The name you want on the Hub
username = "Harsha901"                   # OPTIONAL: It usually detects this auto-magically

# 2. Load the Model & Tokenizer
# =====================================================
print(f"Loading model from {local_model_path}...")

# Load the model (FP32 version is best for the Hub so others can quantize it themselves)
model = AutoModelForSequenceClassification.from_pretrained(local_model_path)
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# 3. Push to Hub
# =====================================================
print(f"Pushing to Hugging Face Hub: {repo_name}...")

# This pushes the weights, config, and vocabulary
# It will create the repo if it doesn't exist
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"\nSuccess! Your model is live at:")
print(f"https://huggingface.co/{username}/{repo_name}")

Loading model from ./distilled_student_saved...
Pushing to Hugging Face Hub: distilbert-sst2-student...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...zeqsqtw/model.safetensors:   3%|2         | 7.05MB /  268MB            

README.md: 0.00B [00:00, ?B/s]


Success! Your model is live at:
https://huggingface.co/Harsha901/distilbert-sst2-student
